In [2]:
import pandas as pd
import json

In [15]:
allprices = json.load(open("../_data/AllPrices.json"))

In [16]:
allprices['data']['00010d56-fe38-5e35-8aed-518019aa36a5']['paper']['tcgplayer']['retail']['foil']['2022-10-28']

8.99

In [97]:
id_list = [i for i in allprices['data']]
len(id_list)

77055

In [94]:
price_list = []
for id in id_list:
  try:
    # only keep instances where there are paper cards sold at tcgplayer at retail price
    card_styles = allprices['data'][id].get('paper').get('tcgplayer').get('retail')
    for each_card in card_styles:
        for each_date in card_styles.get(each_card):
          price = card_styles.get(each_card).get(each_date)
          price_list.append( (id,each_card,each_date,price) )
  except:
    pass

In [95]:
price_df = pd.DataFrame(price_list, columns=['id', 'type', 'date','price'])

In [98]:
len(price_df.id.unique())

66595

In [100]:
price_df.to_csv("../_data/allprices_output.csv", index=False)

In [112]:
avg_price = price_df[['id', 'type', 'price']].groupby(['id', 'type']).mean().round(2).reset_index()
avg_price

,id,type,price
0,00010d56-fe38-5e35-8aed-518019aa36a5,foil,8.69
1,0001e0d0-2dcd-5640-aadc-a84765cf5fc9,normal,5.51
2,0003caab-9ff5-5d1a-bc06-976dd0457f19,foil,4.93
3,0003caab-9ff5-5d1a-bc06-976dd0457f19,normal,0.19
4,0003d249-25d9-5223-af1e-1130f09622a7,foil,0.30
...,...,...,...
100815,fffa9242-6444-567c-a45b-da2b45249232,foil,0.47
100816,fffb659e-b3fa-5cd8-9423-fe5ac74248b5,normal,0.30
100817,fffbc95a-c4d1-56aa-8653-8a7c71fe19ce,normal,7.45
100818,fffc1305-a118-559b-9504-3d7b56ca0bde,normal,1.65


In [113]:
avg_price.to_csv('../_data/avg_price_output.csv', index=False)

In [ ]:
# TODO: connect to the allprintings.json file to get card data